In [18]:
import os, sys, re, json, time, unittest
import itertools, collections
import re
import operator
from sklearn import datasets, linear_model
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [20]:
import nltk
#read in data to clean and separate into training and test set
good = []
bad = []
score = []
pre_tags = []
tags = []
pre_lines = []
a = 0
b = 0
#1 is good
#0 is not
for line in open('listings_scores.csv').readlines():
    data = line.split('\t')
    clean_score = re.sub("[^0-9]", "", data[1])
    desc = data[0]
    if clean_score != '' and int(clean_score) <= 100 and int(clean_score) > 0:
        if int(clean_score) >= 90:
            for each in desc.lower().split('.'):
                if each != ' ':
                    good.append(re.sub('[^A-Za-z]', ' ', each))
                    pre_lines.append((re.sub('[^A-Za-z]', ' ', each), 'good'))
                    score.append(int(clean_score))
                    tags.append(int(1))
                    pre_tags.append(int(1))
                    a += 1
        elif int(clean_score) < 90:
            for each in desc.lower().split('.'):
                if each != ' ':
                    pre_lines.append((re.sub('[^A-Za-z]', ' ', each), 'bad'))
                    bad.append(re.sub('[^A-Za-z]', ' ', each))
                    score.append(int(clean_score))
                    pre_tags.append(int(0))
                    b += 1


In [21]:
print a
print len(good)
print b
print len(bad)
#pre_lines = good + bad
print len(pre_lines)

189616
189616
22763
22763
212379


In [7]:
training = pre_lines[:len(pre_lines)/2]
testing = pre_lines[len(pre_lines)/2:]

tag_train = pre_tags[:len(pre_tags)/2]
tag_test = pre_tags[len(pre_tags)/2:]

print len(training)
print len(tag_train)

106189
106189


In [8]:
feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(pre_lines)
y = pre_tags

X_train = X[:len(pre_lines)/2]
X_test = X[len(pre_lines)/2:]
Y_train = pre_tags[:len(pre_tags)/2]
Y_test = pre_tags[len(pre_tags)/2:]


# Train model
clf_0 = LogisticRegression().fit(X_train, Y_train)
 
# Predict on training set
pred_y_0 = clf_0.predict(X_test)

In [ ]:
print( accuracy_score(pred_y_0, Y_test) )
print( np.unique( pred_y_0 ) )

In [9]:
from sklearn.utils import resample

# Separate majority and minority classes
lines_majority = good
lines_minority = bad
 
# Upsample minority class
bad_upsampled = resample(lines_minority, 
                         replace=True,     # sample with replacement
                         n_samples=189616,    # to match majority class
                         random_state=123) # reproducible results
 
    
# Combine majority class with upsampled minority class
print len(bad_upsampled)
print len(good)

189616
189616


In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from random import shuffle


new_list = []

for each in good:
    new_list.append((each, int(1)))

print len(new_list)
bad_upsampled = list(bad_upsampled)
for each in bad_upsampled:
    new_list.append((each, int(0)))

shuffle(new_list)

new_lines = []
new_tags = []

for each in new_list:
    new_lines.append(each[0])
    new_tags.append(each[1])
                     
                     
print len(new_lines)
print len(new_tags)

189616
379232
379232


In [ ]:
feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(new_lines)
y = new_tags

X_train = X[:len(new_lines)/2]
X_test = X[len(new_lines)/2:]
Y_train = new_tags[:len(new_tags)/2]
Y_test = new_tags[len(new_tags)/2:]


# Train model
clf_0 = LogisticRegression().fit(X_train, Y_train)
 
# Predict on training set
pred_y_0 = clf_0.predict(X_test)

In [ ]:
print( accuracy_score(pred_y_0, Y_test) )
print( np.unique( pred_y_0 ) )

In [ ]:
from sklearn.svm import SVC
feature_extraction = TfidfVectorizer()
X = feature_extraction.fit_transform(pre_lines)
y = pre_tags

X_train = X[:len(pre_lines)/2]
X_test = X[len(pre_lines)/2:]
Y_train = pre_tags[:len(pre_tags)/2]
Y_test = pre_tags[len(pre_tags)/2:]


# Train model
clf_3 = SVC(kernel='linear', 
            class_weight='balanced', # penalize
            probability=True)
 
clf_3.fit(X_train, Y_train)
 
# Predict on training set
pred_y_3 = clf_3.predict(X_test)
 
# Is our model still predicting just one class?
print( np.unique( pred_y_3 ) )
# [0 1]
 
# How's our accuracy?
print( accuracy_score(pred_y_3, Y_test) )
# 0.688

In [15]:
print pre_lines[0]

(' lands  end hideaway light and airy in law with dedicated bathroom and kitchenette', 'good')


In [17]:
# import 
import numpy as np
import pandas as pd
#test = np.array(pre_lines).reshape(212379,2)
heading = [('desc', 'rating')]
together = heading + pre_lines
#print together[1]
colnam = ['desc', 'rating']
df = pd.DataFrame.from_records(pre_lines, columns=colnam)

df.head(3)
df.shape

(212379, 2)

In [32]:
from sklearn import preprocessing

df.columns
# TODO: create a LabelEncoder object and fit it to each feature in X


# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
X_2 = df.apply(le.fit_transform)
X_2.head()

,desc,rating
0,69687,1
1,14583,1
2,105271,1
3,29455,1
4,30757,1


In [38]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example



inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  1.,  0., ...,  0.,  0.,  1.]])

In [23]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

In [24]:
words = []
for text, rating in pre_lines:
    for word in text.split():
        if word.lower() not in stop_words:
            words.append(word.lower())
word_features = nltk.FreqDist(words)
most_common = word_features.most_common()[:10000]
print most_common[:10]

[('room', 22482), ('san', 16135), ('na', 16011), ('francisco', 15567), ('street', 15362), ('bedroom', 14277), ('kitchen', 13880), ('home', 13804), ('house', 12421), ('apartment', 12401)]


In [25]:
def feature_sep(descriptions):
    words = []
    for word in descriptions.split(" "):
        words.append(word)
    features = {}
    for w in most_common:
        features[w] = (w in words)
    return features

In [ ]:
featuresets = [(feature_sep(descriptions), tag) for (descriptions, tag) in pre_lines]

In [ ]:
training_set = featuresets[:len(featuresets)/2]
test_set = featuresets[len(featuresets)/2:]

In [ ]:
classifier = LogisticRegression(training_set)
nltk.
refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(testfeats):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)




In [ ]:
print (nltk.classify.accuracy(classifier, test_set)*100)
classifier.show_most_informative_features(30)

In [ ]:
print (nltk.classify.accuracy(classifier, test_set)*100)
classifier.show_most_informative_features(20)

In [ ]:
import pickle
save_classifier = open("naivebayes.pickle", "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()